In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
file_path = '/content/order-dataset.csv'
data = pd.read_csv(file_path, delimiter=';')
data['completed_at'] = pd.to_datetime(data['completed_at'])
data.dropna(inplace=True)
data['hour'] = data['completed_at'].dt.hour
data['day_of_week'] = data['completed_at'].dt.dayofweek
data['month'] = data['completed_at'].dt.month
data.drop(columns=['completed_at'], inplace=True)


label_encoder = LabelEncoder()
data['operator_id'] = label_encoder.fit_transform(data['operator_id'])


X = data.drop(columns=['operator_id', 'order_id'])
y = data['operator_id']

In [2]:

window_size = 20
generator = TimeseriesGenerator(X, y, length=window_size, batch_size=100)


num_samples = len(generator)


train_size = int(num_samples * 0.7)

train_generator = TimeseriesGenerator(X, y, length=window_size, batch_size=1, end_index=train_size)
test_generator = TimeseriesGenerator(X, y, length=window_size, batch_size=1, start_index=train_size)

In [3]:
# Build the GRU model
model = Sequential()
model.add(LSTM(32, activation='leaky_relu', input_shape=(window_size, X.shape[1]), return_sequences=True))
model.add(LSTM(20, activation='leaky_relu'))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [4]:
# Train the model
model.fit(train_generator, epochs=10, validation_data=test_generator)

Epoch 1/10
1514/1514 [==============================] - 510s 335ms/step - loss: 2.7461 - accuracy: 0.1268 - val_loss: 4.9782 - val_accuracy: 0.0458
Epoch 2/10
1514/1514 [==============================] - 504s 333ms/step - loss: 2.2409 - accuracy: 0.2166 - val_loss: 5.0573 - val_accuracy: 0.0457
Epoch 3/10
1514/1514 [==============================] - 508s 336ms/step - loss: 1.9931 - accuracy: 0.2583 - val_loss: 5.6143 - val_accuracy: 0.0441
Epoch 4/10
1514/1514 [==============================] - 501s 331ms/step - loss: 1.9642 - accuracy: 0.2569 - val_loss: 4.6773 - val_accuracy: 0.0465
Epoch 5/10
1514/1514 [==============================] - 509s 337ms/step - loss: 1.8821 - accuracy: 0.2801 - val_loss: 5.2539 - val_accuracy: 0.0441
Epoch 6/10
1514/1514 [==============================] - 508s 336ms/step - loss: 1.8556 - accuracy: 0.2754 - val_loss: 5.8140 - val_accuracy: 0.0443
Epoch 7/10
1514/1514 [==============================] - 510s 337ms/step - loss: 1.8392 - accuracy: 0.2682 - val_

In [5]:
# Evaluate the model
loss, accuracy = model.evaluate(test_generator)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")


217440/217440 [==============================] - 546s 3ms/step - loss: 8.6243 - accuracy: 0.0499
Test Loss: 8.6243
Test Accuracy: 0.0499
